# Series Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

## Initialisation

Basic approach to determine the project directory

In [10]:
import os
import glob

from datetime import datetime
import time

import json
import jinja2

from PIL import Image

from common import Printable, projdir
from event import Event
from constants import *

## Process Years

Process all available years

In [11]:
events = {}
existingNames = {}

def processEvents():
    '''Process all events from 2010 onwards'''

    eventPaths = sorted(glob.glob(os.path.join(projdir, EVENTS_DIR, '[2][0-9][1-9][0-9]')))
    currYear = datetime.today().year
    for eventPath in eventPaths:
        eventYear = int(os.path.basename(eventPath))

        if eventYear == currYear or appConfig['History']['Refresh']:
            if eventYear == currYear:
                verbosity = appConfig['Latest']['Verbosity']
            else:
                verbosity = appConfig['History']['Verbosity']

            event = Event(eventPath, appConfig, existingNames=existingNames, verbosity=verbosity)
            event.processEvent(runReports=False)

            events[event.year] = event

In [12]:
if __name__ == '__main__':
    pc1 = time.perf_counter()
    
    # Read main config
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        try:
            appConfig = json.loads(jsonTxt)
        except:
            logger = Printable()
            logger.logError('Could not parse {}'.format(filename))
            raise
    
    processEvents()
    
    pc2 = time.perf_counter()
    print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Processing 2010...
All done!

Processing 2011...
All done!

Processing 2012...
All done!

Processing 2013...
All done!

Processing 2014...
All done!

Processing 2015...
All done!

Processing 2016...
All done!

Processing 2017...
All done!

Processing 2018...
All done!

Processing 2019...
All done!

Processing 2020...
All done!

Processing 2021...
All done!

Processing 2022...
All done!

Reports completed in 0.53 seconds


## All Done!